# Global Heat Budget Closure - DAILY FILES - Jacopo Sala

*Contributors*: [Jan-Erik Tesdal](https://github.com/jetesdal), [Ryan Abernathey](https://github.com/rabernat) and [Ian Fenty](https://github.com/ifenty)

A major part of this tutorial is based on "*A Note on Practical Evaluation of Budgets in ECCO Version 4 Release 3"* by Christopher G. Piecuch (https://ecco.jpl.nasa.gov/drive/files/Version4/Release3/doc/v4r3_budgets_howto.pdf). Calculation steps and Python code presented here are converted from the MATLAB code presented in the above reference.

## Objectives

Evaluating and closing the heat budget over the global ocean. 

## Introduction

The ocean heat content (OHC) variability is described here with potential temperature ($\theta$) which is given by the ECCOv4 diagnostic output `THETA`. The budget equation describing the change in $\theta$ is evaluated in general as

$$
\frac{\partial \theta}{\partial t} = -\nabla \cdot (\theta \mathbf{u})-\nabla\cdot\mathbf{F}_\textrm{diff}^{\theta}+{F}_\textrm{forc}^{\theta}
$$

The heat budget includes the change in temperature over time ($\frac{\partial \theta}{\partial t}$), the convergence of heat advection ($-\nabla \cdot (\theta \mathbf{u})$) and heat diffusion ($-\nabla\cdot\mathbf{F}_\textrm{diff}$), plus downward heat flux from the atmosphere (${F}_\textrm{forc}$). Note that in our definition ${F}_\textrm{forc}$ contains both latent and sensible air-sea heat fluxes, longwave and shortwave radiation, as well as geothermal heat flux.

In the special case of ECCOv4, the heat budget is formulated as
$$
\underbrace{\frac{\partial(s^*\theta)}{\partial t}}_{G^{\theta}_\textrm{total}} = \underbrace{-\nabla_{z^{*}} \cdot(s^*\theta\,\mathbf{v}_{res}) - \frac{\partial(\theta\,w_{res})}{\partial z^{*}}}_{G^{\theta}_\textrm{advection}}\underbrace{- s^* ({\nabla\cdot\mathbf{F}_\textrm{diff}^{\theta}})}_{G^{\theta}_\textrm{diffusion}} + \underbrace{s^* {F}_\textrm{forc}^{\theta}}_{G^{\theta}_\textrm{forcing}}
$$

where $z^{*} = \frac{z - \eta}{H + \eta}H$ and $\nabla_{z^{*}}$/$\frac{\partial}{\partial z^{*}}$ are horizontal/vertical divergences in the $z^*$ frame. Also note that the advection is now separated into horizontal ($\mathbf{v}_{res}$) and vertical ($w_{res}$) components, and there is a scaling factor ($s^* = 1+ \frac{\eta}{H}$) applied to the horizontal advection as well as the diffusion term ($G^{\theta}_\textrm{diffusion}$) and forcing term ($G^{\theta}_\textrm{forcing}$). $s^*$ is a function of $\eta$ which is the displacement of the ocean surface from its resting position of $z=0$ (i.e., sea height anomaly). $H$ is the ocean depth. $s^{*}$ comes from the coordinate transformation from z to $z^*$ (Campin and Adcroft, 2004; Campin et al., 2004). See [ECCOv4 Global Volume Budget Closure](https://ecco-v4-python-tutorial.readthedocs.io/ECCO_v4_Volume_budget_closure.html#ECCOv4-Global-Volume-Budget-Closure) for a more detailed explanation of the $z^*$ coordinate system.

Note that the velocity terms in the ECCOv4 heat budget equation ($\mathbf{v}_{res}$ and $w_{res}$) are described as the "residual mean" velocities, which contain both the resolved (Eulerian) flow field, as well as the "GM bolus" velocity (i.e., parameterizing unresolved eddy effects):

$$(u_{res},v_{res},w_{res})= (u,v,w)+ (u_b,v_b,w_b)$$

Here $(u_b,v_b,w_b)$ is the bolus velocity parameter, taking into account the correlation between velocity and thickness (also known as the eddy induced transportor the eddy advection term).

## Evaluating the heat budget

We will evalute each term in the above heat budget 

$$G^{\theta}_\textrm{total} = G^{\theta}_\textrm{advection} + G^{\theta}_\textrm{diffusion} + G^{\theta}_\textrm{forcing}$$

The total tendency of $\theta$ ($G^{\theta}_\textrm{total}$) is the sum of the $\theta$ tendencies from advective heat convergence ($G^{\theta}_\textrm{advection}$), diffusive heat convergence ($G^{\theta}_\textrm{diffusion}$) and total forcing ($G^{\theta}_\textrm{forcing}$). 

We present calculation sequentially for each term starting with $G^{\theta}_\textrm{total}$ which will be derived by differencing instantaneous monthly snapshots of $\theta$. The terms on the right hand side of the heat budget are derived from monthly-averaged fields.

## Datasets to download

If you don't have any of the following datasets already, you will need to download them to complete the tutorial. Aside from the grid geometry file (which has no time dimension), you will need 2 monthly datasets for the full time span of ECCOv4r4 output (1992 through 2017). The ShortNames of the datasets are:

- **ECCO_L4_GEOMETRY_LLC0090GRID_V4R4**
- **ECCO_L4_OCEAN_3D_TEMPERATURE_FLUX_LLC0090GRID_MONTHLY_V4R4** (1993-2016)
- **ECCO_L4_HEAT_FLUX_LLC0090GRID_MONTHLY_V4R4** (1993-2016)
- **ECCO_L4_SSH_LLC0090GRID_SNAPSHOT_V4R4** (1993/1/1-2017/1/1, 1st of each month)
- **ECCO_L4_TEMP_SALINITY_LLC0090GRID_SNAPSHOT_V4R4** (1993/1/1-2017/1/1, 1st of each month)

If you haven't yet been through the [download tutorial](https://ecco-v4-python-tutorial.readthedocs.io/Downloading_ECCO_Datasets_from_PODAAC_Python.html) or used the [ecco_download module](https://ecco-v4-python-tutorial.readthedocs.io/Downloading_ECCO_Datasets_from_PODAAC_Python.html#ECCO_download-module), it may help you to review that information before downloading the datasets. If you are downloading the snapshots, you may also find the [snaps_monthly_textlist](https://ecco-v4-python-tutorial.readthedocs.io/ECCO_v4_Salt_and_salinity_budget.html#Load-monthly-snapshots) function helpful; it generates a file list of only the snapshot files for the 1st of each month, which you can then download using `wget`.

## Prepare environment and load ECCOv4 diagnostic output

### Import relevant Python modules

In [1]:
import sys
sys.path.append('/home/jacoposala/ECCOv4-py')
import ecco_v4_py as ecco

In [2]:
import numpy as np
import xarray as xr
import sys
import glob

In [3]:
# Suppress warning messages for a cleaner presentation
import warnings
warnings.filterwarnings('ignore')

In [4]:
# # setting up a dask LocalCluster
# from dask.distributed import Client
# from dask.distributed import LocalCluster
# cluster = LocalCluster()
# client = Client(cluster)
# client

In [5]:
## Import the ecco_v4_py library into Python
## =========================================
##    If ecco_v4_py is not installed in your local Python library, 
##    tell Python where to find it.  The example below adds
##    ecco_v4_py to the user's path if it is stored in the folder
##    ECCOv4-py under the user's home directory

from os.path import join,expanduser,exists,split
user_home_dir = expanduser('~')

sys.path.append(join(user_home_dir,'ECCOv4-py'))

import ecco_v4_py as ecco

In [6]:
# Plotting
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
# ADD TAG TO DO PLOTS
do_plots = False #True #False  # True

### Add relevant constants

In [8]:
# Seawater density (kg/m^3)
rhoconst = 1029
## needed to convert surface mass fluxes to volume fluxes

# Heat capacity (J/kg/K)
c_p = 3994

# Constants for surface heat penetration (from Table 2 of Paulson and Simpson, 1977)
R = 0.62
zeta1 = 0.6
zeta2 = 20.0

### Load ecco_grid

In [9]:
## Set top-level file directory for the ECCO NetCDF files
## =================================================================

## currently set to ~/Downloads/ECCO_V4r4_PODAAC, 
## the default if ecco_podaac_download was used to download dataset granules
# ECCO_dir = join(user_home_dir,'Downloads','ECCO_V4r4_PODAAC')

ECCO_dir = '/Users/jacoposala/Desktop/CU/3.RESEARCH/NASA_project/NEW_heatBudgetECCO/data/'

In [10]:
## Load the model grid
# ecco_grid = xr.open_dataset(glob.glob(join(ECCO_dir,'*GEOMETRY*','*.nc'))[0])
ecco_grid = xr.open_dataset(ECCO_dir + 'eccov4r4_grid/GRID_GEOMETRY_ECCO_V4r4_native_llc0090.nc')

### Volume
Calculate the volume of each grid cell. This is used when converting advective and diffusive flux convergences and calculating volume-weighted averages.

In [11]:
# Volume (m^3)
vol = (ecco_grid.rA*ecco_grid.drF*ecco_grid.hFacC).transpose('tile','k','j','i')

### Load daily snapshots

In [ ]:
folder_year = '1992'

In [ ]:
ECCO_snap_dir = ECCO_dir + 'eccov4r4_daily_snap_' + folder_year + '/'

In [ ]:
# https://search.earthdata.nasa.gov/search/granules?portal=podaac&p=C1991543817-POCLOUD&pg[0][v]=f&pg[0][gsk]=-start_date&q=ecco%20snapshot&tl=1694750694.602!3!!
# https://search.earthdata.nasa.gov/search/granules?portal=podaac&p=C1991543757-POCLOUD&pg[0][v]=f&pg[0][gsk]=-start_date&q=ecco%20snapshot&tl=1694750694.602!3!!
# https://search.earthdata.nasa.gov/search/granules?p=C1991543812-POCLOUD&pg[0][v]=f&pg[0][gsk]=-start_date&q=ecco%203d&tl=1694750676.034!3!!

                      ###################### FILE NAMES FOR EARTHDATA ######################
# jsala
# Salinity1996

############## DAILY FILES
# ECCO Ocean and Sea-Ice Surface Heat Fluxes - Daily Mean llc90 Grid (Version 4 Release 4)
# ECCO Ocean Three-Dimensional Potential Temperature Fluxes - Daily Mean llc90 Grid (Version 4 Release 4)

############## DAILY SNAPSHOTS
# ECCO Sea Surface Height - Snapshot llc90 Grid (Version 4 Release 4)
# ECCO Ocean Temperature and Salinity - Snapshot llc90 Grid (Version 4 Release 4)

# 1 SNAPSHOT MORE (COMPARED TO MEAN FILES) IS NEEDED    
    

In [ ]:
year_start = 1992
year_end = 2018

# open ETAN and THETA snapshots (beginning of each month)
ecco_monthly_SSH = xr.open_mfdataset(join(ECCO_snap_dir, 'SEA_SURFACE_HEIGHT_snap*.nc'),\
                                       data_vars='minimal',coords='minimal',compat='override')
ecco_monthly_TS = xr.open_mfdataset(join(ECCO_snap_dir,'OCEAN_TEMPERATURE_SALINITY_snap*.nc'),\
                                       data_vars='minimal',coords='minimal',compat='override')
ecco_monthly_snaps = xr.merge((ecco_monthly_SSH['ETAN'],ecco_monthly_TS['THETA']))


# time mask for snapshots
time_snap_mask = np.logical_and(ecco_monthly_snaps.time.values >= np.datetime64(str(year_start)+'-01-01','ns'),\
                                ecco_monthly_snaps.time.values < np.datetime64(str(year_end)+'-01-02','ns'))

ecco_monthly_snaps = ecco_monthly_snaps.isel(time=time_snap_mask)

In [ ]:
# 1993-01 (beginning of first month) to 2015-01-01 (end of last month, 2014-12)
print(ecco_monthly_snaps.ETAN.time.isel(time=[0, -1]).values)

In [ ]:
# Find the record of the last snapshot
## This is used to defined the exact period for monthly mean data 
last_record_date = ecco.extract_yyyy_mm_dd_hh_mm_ss_from_datetime64(ecco_monthly_snaps.time[-1].values)
print(last_record_date)

### Load daily mean data 

In [ ]:
ECCO_daily_dir = ECCO_dir + 'eccov4r4_daily_mean_' + folder_year + '/'

In [ ]:
## Open ECCO monthly mean variables
ecco_vars_int = xr.open_mfdataset(join(ECCO_daily_dir, 'OCEAN_3D*.nc'),\
                                       data_vars='minimal',coords='minimal',compat='override')
ecco_vars_sfc = xr.open_mfdataset(join(ECCO_daily_dir,'*_HEAT_FLUX*.nc'),\
                                       data_vars='minimal',coords='minimal',compat='override')
ecco_monthly_mean = xr.merge((ecco_vars_int,\
                              ecco_vars_sfc[['TFLUX','oceQsw']]))

# time mask for monthly means
time_mean_mask = np.logical_and(ecco_monthly_mean.time.values >= np.datetime64(str(year_start)+'-01-01','ns'),\
                                ecco_monthly_mean.time.values < np.datetime64(str(year_end)+'-01-01','ns'))

ecco_monthly_mean = ecco_monthly_mean.isel(time=time_mean_mask)

In [ ]:
# Print first and last time points of the monthly-mean records
print(ecco_monthly_mean.time.isel(time=[0, -1]).values)

Each monthly mean record is bookended by a snapshot. We should have one more snapshot than monthly mean record.

In [ ]:
print('Number of monthly mean records: ', len(ecco_monthly_mean.time))
print('Number of monthly snapshot records: ', len(ecco_monthly_snaps.time))

In [ ]:
# Drop superfluous coordinates (We already have them in ecco_grid)
ecco_monthly_mean = ecco_monthly_mean.reset_coords(drop=True)

### Merge dataset of monthly mean and snapshots data
Merge the two datasets to put everything into one single dataset

In [ ]:
ds = xr.merge([ecco_monthly_mean,
               ecco_monthly_snaps.rename({'time':'time_snp','ETAN':'ETAN_snp', 'THETA':'THETA_snp'})])

### Create the xgcm 'grid' object

The `xgcm` 'grid' object is used to calculate the flux divergences across different tiles of the lat-lon-cap grid and the time derivatives from ``THETA`` snapshots

In [ ]:
# Change time axis of the snapshot variables
ds.time_snp.attrs['c_grid_axis_shift'] = 0.5

In [ ]:
grid = ecco.get_llc_grid(ds)

### Number of seconds in each month
The xgcm `grid` object includes information on the time axis, such that we can use it to get $\Delta t$, which is the time span between the beginning and end of each month (in seconds).

In [ ]:
delta_t = grid.diff(ds.time_snp, 'T', boundary='fill', fill_value=np.nan)

# Convert to seconds
delta_t = delta_t.astype('f4') / 1e9

## Calculate total tendency of $\theta$ ($G^{\theta}_\textrm{total}$)

We calculate the monthly-averaged time tendency of ``THETA`` by differencing monthly ``THETA`` snapshots. Remember  that we need to include a scaling factor due to the nonlinear free surface formulation. Thus, we need to use snapshots of both `ETAN` and `THETA`  to evaluate $s^*\theta$.

In [ ]:
# Calculate the s*theta term
sTHETA = ds.THETA_snp*(1+ds.ETAN_snp/ecco_grid.Depth)

In [ ]:
# Total tendency (psu/s)
G_total = sTHETA.diff(dim='time_snp')/np.expand_dims(delta_t.values,axis=(1,2,3,4))

# re-assign and rename time coordinate
G_total = G_total.rename({'time_snp':'time'})
G_total = G_total.assign_coords({'time':delta_t.time.values})


> **Note**: Unlike the monthly snapshots `ETAN_snp` and `THETA_snp`, the resulting data array `G_total` has now the same time values as the time-mean fields (middle of the month).

### Plot the time-mean $\partial \theta / \partial t$, total $\Delta \theta$, and one example $\partial \theta / \partial t$ field

####  Time-mean $\partial \theta / \partial t$
The time-mean $\partial \theta / \partial t$ (i.e., $\overline{G^{\theta}_\textrm{total}}$), is given by 

$\overline{G^{\theta}_\textrm{total}} = \sum_{i=1}^{nm} w_i G^{\theta}_\textrm{total}$

with $\sum_{i=1}^{nm} w_i = 1$ and  nm=number of months

In [ ]:
# The weights are just the number of seconds per month divided by total seconds
month_length_weights = delta_t / delta_t.sum()

In [ ]:
if do_plots:
    # The weighted mean weights by the length of each month (in seconds)
    G_total_mean = (G_total*month_length_weights).sum('time').compute()

In [ ]:
if do_plots:
    plt.figure(figsize=(15,15))

    for idx, k in enumerate([0,10,25]):
        p = ecco.plot_proj_to_latlon_grid(ecco_grid.XC, ecco_grid.YC, G_total_mean.isel(k=k),show_colorbar=True,
                                          cmap='RdBu_r', user_lon_0=-67, dx=2, dy=2, subplot_grid=[3,1,idx+1]);
        p[1].set_title(r'$\overline{G^\theta_{total}}$ at z = %i m (k = %i) [$^\circ$C s$^{-1}$]'\
                       %(np.round(-ecco_grid.Z[k].values),k), fontsize=16)

#### Total $\Delta \theta$

How much did ``THETA`` change over the analysis period?

In [ ]:
# The number of seconds in the entire period 
seconds_in_entire_period = \
    float(ds.time_snp[-1] - ds.time_snp[0])/1e9
print ('seconds in analysis period: ', seconds_in_entire_period)

# which is also the sum of the number of seconds in each month
print('Sum of seconds in each month ', delta_t.sum().values)

In [ ]:
if do_plots:
    THETA_delta = G_total_mean*seconds_in_entire_period

In [ ]:
if do_plots:
    plt.figure(figsize=(15,5));
    ecco.plot_proj_to_latlon_grid(ecco_grid.XC, ecco_grid.YC, \
                                  THETA_delta.isel(k=0),show_colorbar=True,\
                                  cmin=-4, cmax=4, \
                                  cmap='RdBu_r', user_lon_0=-67, dx=0.2, dy=0.2);
    plt.title(r'Predicted $\Delta \theta$ at the sea surface [$^\circ$C] from $\overline{G^\theta_{total}}$',fontsize=16);

We can sanity check the total ``THETA`` change that we found by multipling the time-mean ``THETA`` tendency with the number of seconds in the simulation by comparing that with the difference in ``THETA`` between the end of the last month and start of the first month. 

In [ ]:
THETA_delta_method_2 = ds.THETA_snp.isel(time_snp=-1) - ds.THETA_snp.isel(time_snp=0)

In [ ]:
if do_plots:
    plt.figure(figsize=(15,5));
    ecco.plot_proj_to_latlon_grid(ecco_grid.XC, ecco_grid.YC, \
                                  THETA_delta_method_2.isel(k=0),show_colorbar=True,\
                                  cmin=-4, cmax=4, \
                                  cmap='RdBu_r', user_lon_0=-67, dx=0.2, dy=0.2);
    plt.title(r'Actual $\Delta \theta$ [$^\circ$C]', fontsize=16);

#### Example $G^\theta_{total}$ field at a particular time

In [ ]:
# get an array of YYYY, MM, DD, HH, MM, SS for 
#dETAN_dT_perSec at time index 100
curr_t_ind = 2
tmp = str(G_total.time.values[curr_t_ind])
print(tmp)

In [ ]:
if do_plots:
    plt.figure(figsize=(15,5));
    ecco.plot_proj_to_latlon_grid(ecco_grid.XC, ecco_grid.YC, G_total.isel(time=curr_t_ind,k=0), show_colorbar=True,
                                  cmap='RdBu_r', user_lon_0=-67, dx=0.2, dy=0.2);

    plt.title(r'$G^\theta_{total}$ at the sea surface [$^\circ$C s$^{-1}$] during ' + 
              str(tmp)[0:4] +'/' + str(tmp)[5:7], fontsize=16);

For any given month the time rate of change of ``THETA`` is strongly dependent on the season. In the above we are looking at May 2001. We see positive ``THETA`` tendency in the northern hemisphere and cooling in the southern hemisphere.

## Calculate tendency due to advective convergence ($G^{\theta}_\textrm{advection}$)
### Horizontal convergence of advective heat flux
The relevant fields from the diagnostic output here are
- `ADVx_TH`: U Component Advective Flux of Potential Temperature (degC m^3/s)
- `ADVy_TH`: V Component Advective Flux of Potential Temperature (degC m^3/s)

The xgcm `grid` object is then used to take the convergence of the horizontal heat advection.

> **Note**: when using at least one recent version of `xgcm` (v0.8.1), errors were triggered when calling `diff_2d_vector` below. If you have this issue, as a temporary fix you can locate the `xgcm` directory
> 
> ``` python
> import xgcm
> print(xgcm.__path__)
> ```
> 
> and then substitute `padding.py` and `grid_ufunc.py` with the files linked below:
> 
> [padding.py](https://github.com/ECCO-GROUP/ECCO-v4-Python-Tutorial/raw/master/misc/xgcm_fixes/padding.py)
> 
> [grid_ufunc.py](https://github.com/ECCO-GROUP/ECCO-v4-Python-Tutorial/raw/master/misc/xgcm_fixes/grid_ufunc.py)
> 
> Alternatively, you can revert `xgcm` to an earlier version (e.g., v0.6.0) that should not have this issue.

In [ ]:
# Set fluxes on land to zero (instead of NaN)
ds['ADVx_TH'] = ds.ADVx_TH.where(ecco_grid.hFacW.values > 0,0)
ds['ADVy_TH'] = ds.ADVy_TH.where(ecco_grid.hFacS.values > 0,0)

# compute horizontal components of divergence
ADVxy_diff = grid.diff_2d_vector({'X' : ds.ADVx_TH, 'Y' : ds.ADVy_TH}, boundary = 'fill')

# Convergence of horizontal advection (degC m^3/s)
adv_hConvH = (-(ADVxy_diff['X'] + ADVxy_diff['Y']))

### Vertical convergence of advective heat flux
The relevant field from the diagnostic output is
- `ADVr_TH`: Vertical Advective Flux of Potential Temperature (degC m^3/s)

In [ ]:
# Set fluxes on land to zero (instead of NaN)
ds['ADVr_TH'] = ds.ADVr_TH.where(ecco_grid.hFacC.values > 0,0)

# Load monthly averages of vertical advective flux
ADVr_TH = ds.ADVr_TH.transpose('time','tile','k_l','j','i')

> **Note**: For `ADVr_TH`, `DFrE_TH` and `DFrI_TH`, we need to make sure that sequence of dimensions are consistent. When loading the fields use `.transpose('time','tile','k_l','j','i')`. Otherwise, the divergences will be not correct (at least for `tile = 12`).

In [ ]:
# Convergence of vertical advection (degC m^3/s)
adv_vConvH = grid.diff(ADVr_TH, 'Z', boundary='fill')

> **Note**: In case of the volume budget (and salinity conservation), the surface forcing (`oceFWflx`) is already included at the top level (`k_l = 0`) in ``WVELMASS``.  Thus, to keep the surface forcing term explicitly represented, one needs to zero out the values of ``WVELMASS`` at the surface so as to avoid double counting (see the [Volume budget closure](https://ecco-v4-python-tutorial.readthedocs.io/ECCO_v4_Volume_budget_closure.html) tutorial). This is not the case for the heat budget. `ADVr_TH` does not include the sea surface forcing. Thus, the vertical advective flux (at the air-sea interface) should not be zeroed out.

### Total convergence of advective flux ($G^{\theta}_\textrm{advection}$)
We can get the total convergence by simply adding the horizontal and vertical component. 

In [ ]:
# Sum horizontal and vertical convergences and divide by volume (degC/s)
G_advection = (adv_hConvH + adv_vConvH)/vol

### Plot the time-mean $G^{\theta}_\textrm{advection}$

In [ ]:
if do_plots:
    G_advection_mean = (G_advection*month_length_weights).sum('time').compute()

In [ ]:
if do_plots:
    plt.figure(figsize=(15,15))

    for idx, k in enumerate([0,1,25]):
        p = ecco.plot_proj_to_latlon_grid(ecco_grid.XC, ecco_grid.YC, G_advection_mean.isel(k=k),show_colorbar=True,
                                          cmin=-1e-6, cmax=1e-6, cmap='RdBu_r', user_lon_0=-67, dx=2, dy=2, 
                                          subplot_grid=[3,1,idx+1]);
        p[1].set_title(r'$\overline{G^\theta_{advection}}$ at z = %i m (k = %i) [$^\circ$C s$^{-1}$]'\
                       %(np.round(-ecco_grid.Z[k].values),k), fontsize=16)

### Example $G^{\theta}_\textrm{advection}$ field at a particular time

In [ ]:
curr_t_ind = 2
tmp = str(G_advection.time.values[curr_t_ind])
print(tmp)

In [ ]:
if do_plots:
    plt.figure(figsize=(15,5));

    ecco.plot_proj_to_latlon_grid(ecco_grid.XC, ecco_grid.YC, G_advection.isel(time=curr_t_ind,k=k), show_colorbar=True,
                                  cmin=-1e-6, cmax=1e-6, cmap='RdBu_r', user_lon_0=-67, dx=0.2, dy=0.2)
    plt.title(r'$G^\theta_{advection}$ at the sea surface [$^\circ$C s$^{-1}$] during ' + 
              str(tmp)[0:4] +'/' + str(tmp)[5:7], fontsize=16)
    plt.show()

## Calculate tendency due to diffusive convergence ($G^{\theta}_\textrm{diffusion}$)
### Horizontal convergence of advective heat flux
The relevant fields from the diagnostic output here are
- `DFxE_TH`: U Component Diffusive Flux of Potential Temperature (degC m^3/s)
- `DFyE_TH`: V Component Diffusive Flux of Potential Temperature (degC m^3/s)

As with advective fluxes, we use the xgcm `grid` object to calculate the convergence of horizontal heat diffusion.

In [ ]:
# Set fluxes on land to zero (instead of NaN)
ds['DFxE_TH'] = ds.DFxE_TH.where(ecco_grid.hFacW.values > 0,0)
ds['DFyE_TH'] = ds.DFyE_TH.where(ecco_grid.hFacS.values > 0,0)

DFxyE_diff = grid.diff_2d_vector({'X' : ds.DFxE_TH, 'Y' : ds.DFyE_TH}, boundary = 'fill')

# Convergence of horizontal diffusion (degC m^3/s)
dif_hConvH = (-(DFxyE_diff['X'] + DFxyE_diff['Y']))

### Vertical convergence of advective heat flux
The relevant fields from the diagnostic output are
- `DFrE_TH`: Vertical Diffusive Flux of Potential Temperature (Explicit part) (degC m^3/s)
- `DFrI_TH`: Vertical Diffusive Flux of Potential Temperature (Implicit part) (degC m^3/s)
> **Note**: Vertical diffusion has both an explicit (`DFrE_TH`) and an implicit (`DFrI_TH`) part.

In [ ]:
# Set land values to zero (not NaN)
ds['ADVr_TH'] = ds.ADVr_TH.where(ecco_grid.hFacC.values > 0,0)

# Set fluxes on land to zero (instead of NaN)
ds['DFrE_TH'] = ds.DFrE_TH.where(ecco_grid.hFacC.values > 0,0)
ds['DFrI_TH'] = ds.DFrI_TH.where(ecco_grid.hFacC.values > 0,0)

# Load monthly averages of vertical diffusive fluxes
DFrE_TH = ds.DFrE_TH.transpose('time','tile','k_l','j','i')
DFrI_TH = ds.DFrI_TH.transpose('time','tile','k_l','j','i')

# Convergence of vertical diffusion (degC m^3/s)
dif_vConvH = grid.diff(DFrE_TH, 'Z', boundary='fill') + grid.diff(DFrI_TH, 'Z', boundary='fill')

### Total convergence of diffusive flux ($G^{\theta}_\textrm{diffusion}$)

In [ ]:
# Sum horizontal and vertical convergences and divide by volume (degC/s)
G_diffusion = (dif_hConvH + dif_vConvH)/vol

### Plot the time-mean $G^{\theta}_\textrm{diffusion}$

In [ ]:
if do_plots:
    G_diffusion_mean = (G_diffusion*month_length_weights).sum('time').compute()

In [ ]:
if do_plots:
    plt.figure(figsize=(15,15))

    for idx, k in enumerate([0,1,25]):
        p = ecco.plot_proj_to_latlon_grid(ecco_grid.XC, ecco_grid.YC, G_diffusion_mean.isel(k=k),show_colorbar=True,
                                          cmin=-3e-6, cmax=3e-6, cmap='RdBu_r', user_lon_0=-67, dx=2, dy=2, 
                                          subplot_grid=[3,1,idx+1]);
        p[1].set_title(r'$\overline{G^\theta_{diffusion}}$ at z = %i m (k = %i) [$^\circ$C s$^{-1}$]'\
                       %(np.round(-ecco_grid.Z[k].values),k), fontsize=16)

### Example $G^{\theta}_\textrm{diffusion}$ field at a particular time

In [ ]:
curr_t_ind = 2
tmp = str(G_diffusion.time.values[curr_t_ind])
print(tmp)

In [ ]:
if do_plots:
    plt.figure(figsize=(15,5));

    ecco.plot_proj_to_latlon_grid(ecco_grid.XC, ecco_grid.YC, G_diffusion.isel(time=curr_t_ind,k=0),show_colorbar=True,
                                  cmin=-3e-6, cmax=3e-6, cmap='RdBu_r', user_lon_0=-67, dx=0.2, dy=0.2)
    plt.title(r'$G^\theta_{diffusion}$ at the sea surface [$^\circ$C s$^{-1}$] during ' + 
              str(tmp)[0:4] +'/' + str(tmp)[5:7], fontsize=16)
    plt.show()

## Calculate tendency due to forcing ($G^{\theta}_\textrm{forcing}$)
Finally, we evaluate the local forcing term due to surface heat and geothermal fluxes.

### Surface heat flux
For the surface contribution, there are two relevant model diagnostics:
- `TFLUX`: total heat flux (match heat-content variations) (W/m^2)
- `oceQsw`: net Short-Wave radiation (+=down) (W/m^2)

#### Defining terms needed for evaluating surface heat forcing

In [ ]:
Z = ecco_grid.Z.compute()
RF = np.concatenate([ecco_grid.Zp1.values[:-1],[np.nan]])

>**Note**: `Z` and `Zp1` are used in deriving surface heat penetration. MATLAB code uses `RF` from `mygrid` structure.

In [ ]:
q1 = R*np.exp(1.0/zeta1*RF[:-1]) + (1.0-R)*np.exp(1.0/zeta2*RF[:-1])
q2 = R*np.exp(1.0/zeta1*RF[1:]) + (1.0-R)*np.exp(1.0/zeta2*RF[1:])

In [ ]:
# Correction for the 200m cutoff
zCut = np.where(Z < -200)[0][0]
q1[zCut:] = 0
q2[zCut-1:] = 0

In [ ]:
# Create xarray data arrays
q1 = xr.DataArray(q1,coords=[Z.k],dims=['k'])
q2 = xr.DataArray(q2,coords=[Z.k],dims=['k'])

#### Compute vertically penetrating flux
Given the penetrating nature of the shortwave term, to properly evaluate the local forcing term, `oceQsw` must be removed from `TFLUX` (which contains the net latent, sensible, longwave, and shortwave contributions) and redistributed vertically.

In [ ]:
## Land masks
# Make copy of hFacC
mskC = ecco_grid.hFacC.copy(deep=True).compute()

# Change all fractions (ocean) to 1. land = 0
mskC.values[mskC.values>0] = 1

In [ ]:
# Shortwave flux below the surface (W/m^2)
forcH_subsurf = ((q1*(mskC==1)-q2*(mskC.shift(k=-1)==1))*ds.oceQsw).transpose('time','tile','k','j','i')

In [ ]:
# Surface heat flux (W/m^2)
forcH_surf = ((ds.TFLUX - (1-(q1[0]-q2[0]))*ds.oceQsw)\
              *mskC[0]).transpose('time','tile','j','i').assign_coords(k=0).expand_dims('k')

In [ ]:
# Full-depth sea surface forcing (W/m^2)
forcH = xr.concat([forcH_surf,forcH_subsurf[:,:,1:]], dim='k').transpose('time','tile','k','j','i')

### Geothermal flux

The geothermal flux contribution is not accounted for in any of the standard model diagnostics provided as output. Rather, this term, which is time invariant, is provided in the input file `geothermalFlux.bin` contained in the ancillary data archive. This archive can be downloaded from [PO.DAAC](https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/ECCO_L4_ANCILLARY_DATA_V4R4/ancillary_data_input_forcing_ECCO_V4r4.tar.gz), though accessing it there requires downloading a much larger tarball of files (~192 GB). So, for the time being, the `geothermalFlux.bin` file is also stored on the tutorial Github and can be downloaded [here](https://github.com/ECCO-GROUP/ECCO-v4-Python-Tutorial/raw/master/misc/geothermalFlux.bin).

> **Note**: The code cell below assumes `geothermalFlux.bin` has been placed in `~/Downloads` after downloading. Change the directory as needed.

In [ ]:
# Load the geothermal heat flux using the routine 'read_llc_to_tiles'.
# geoflx_dir = join(user_home_dir,'Downloads')
geoflx_dir = ECCO_dir 
geoflx = ecco.read_llc_to_tiles(geoflx_dir, 'eccov4r4_grid/geothermalFlux.bin')

The geothermal flux dataset needs to be saved as an xarray data array with the same format as the model output.

In [ ]:
# Convert numpy array to an xarray DataArray with matching dimensions as the monthly mean fields
geoflx_llc = xr.DataArray(geoflx,coords={'tile': ecco_monthly_mean.tile.values,
                                         'j': ecco_monthly_mean.j.values,
                                         'i': ecco_monthly_mean.i.values},dims=['tile','j','i'])

In [ ]:
if do_plots:
    plt.figure(figsize=(15,5));

    ecco.plot_proj_to_latlon_grid(ecco_grid.XC, ecco_grid.YC, geoflx_llc,show_colorbar=True,cmap='magma', 
                                  user_lon_0=-67, dx=0.2, dy=0.2)
    plt.title(r'Geothermal heat flux [W m$^{-2}$]', fontsize=16)
    plt.show()

Geothermal flux needs to be a three dimensional field since the sources are distributed along the ocean floor at various depths. This requires a three dimensional mask.

In [ ]:
# Create 3d bathymetry mask
mskC_shifted = mskC.shift(k=-1)

mskC_shifted.values[-1,:,:,:] = 0
mskb = mskC - mskC_shifted

# Create 3d field of geothermal heat flux
geoflx3d = geoflx_llc * mskb.transpose('k','tile','j','i')
GEOFLX = geoflx3d.transpose('k','tile','j','i')
GEOFLX.attrs = {'standard_name': 'GEOFLX','long_name': 'Geothermal heat flux','units': 'W/m^2'}

### Total forcing ($G^{\theta}_\textrm{forcing}$)

In [ ]:
# Add geothermal heat flux to forcing field and convert from W/m^2 to degC/s
G_forcing = ((forcH + GEOFLX)/(rhoconst*c_p))/(ecco_grid.hFacC*ecco_grid.drF)

### Plot the time-mean $G^{\theta}_\textrm{forcing}$

In [ ]:
G_forcing_mean = (G_forcing*month_length_weights).sum('time')

In [ ]:
if do_plots:
    plt.figure(figsize=(15,15))

    for idx, k in enumerate([0,1,25]):
        p = ecco.plot_proj_to_latlon_grid(ecco_grid.XC, ecco_grid.YC, G_forcing_mean.isel(k=k),show_colorbar=True,
                                          cmin=-3e-6, cmax=3e-6, cmap='RdBu_r', user_lon_0=-67, dx=2, dy=2, 
                                          subplot_grid=[3,1,idx+1]);
        p[1].set_title(r'$\overline{G^\theta_{forcing}}$ at z = %i m (k = %i) [$^\circ$C s$^{-1}$]'\
                       %(np.round(-ecco_grid.Z[k].values),k), fontsize=16)

$\overline{G^\theta_{forcing}}$ is focused at the sea surface and much smaller (essentially zero) at depth. $\overline{G^\theta_{forcing}}$ is negative for most of the ocean (away from the equator). The spatial pattern in the surface forcing is the same as for diffusion but with opposite sign (see maps for $\overline{G^\theta_{diffusion}}$ above). This makes sense as forcing is to a large extent balanced by diffusion within the mixed layer.

### Example $G^{\theta}_\textrm{forcing}$ field at a particular time

In [ ]:
curr_t_ind = 2
tmp = str(G_advection.time.values[curr_t_ind])
print(tmp)

In [ ]:
if do_plots:
    plt.figure(figsize=(15,5));

    ecco.plot_proj_to_latlon_grid(ecco_grid.XC, ecco_grid.YC, G_forcing.isel(time=curr_t_ind,k=0),show_colorbar=True,
                                  cmin=-5e-6, cmax=5e-6, cmap='RdBu_r', user_lon_0=-67, dx=0.2, dy=0.2)
    plt.title(r'$G^\theta_{forcing}$ at the sea surface [$^\circ$C s$^{-1}$] during ' + 
              str(tmp)[0:4] +'/' + str(tmp)[5:7], fontsize=16)
    plt.show()

### Create new variables - DATA term - ADDED BY JACOPO

In [ ]:
ECCO_daily_DATA_dir = ECCO_dir + 'eccov4r4_daily_mean_DATA_' + folder_year + '/'

In [ ]:
year_start = 1992
year_end = 2018

## Open ECCO monthly mean variables
ecco_monthly_mean = xr.open_mfdataset(join(ECCO_daily_DATA_dir, '*_TEMPERATURE_SALINITY_day_mean*.nc'),\
                                       data_vars='minimal',coords='minimal',compat='override')
# ecco_vars_sfc = xr.open_mfdataset(join(ECCO_daily_dir,'*_HEAT_FLUX*.nc'),\
#                                        data_vars='minimal',coords='minimal',compat='override')
# ecco_monthly_mean = xr.merge((ecco_vars_int,\
#                               ecco_vars_sfc[['TFLUX','oceQsw']]))

# time mask for monthly means
time_mean_mask = np.logical_and(ecco_monthly_mean.time.values >= np.datetime64(str(year_start)+'-01-01','ns'),\
                                ecco_monthly_mean.time.values < np.datetime64(str(year_end)+'-01-01','ns'))

ecco_monthly_mean = ecco_monthly_mean.isel(time=time_mean_mask)
DATA = ecco_monthly_mean.THETA
# Drop superfluous coordinates (We already have them in ecco_grid)
DATA = DATA.reset_coords(drop=True)

### Create new variables - horizontal, vertical terms - ADDED BY JACOPO

In [ ]:
# Advection
adv_hConv = adv_hConvH/vol

adv_vConv = adv_vConvH/vol

In [ ]:
# Diffusion
dif_hConv = dif_hConvH/vol

dif_vConv = dif_vConvH/vol

### Create new variables - zonal, meridional terms - ADDED BY JACOPO

In [ ]:
# Advection
adv_zonal_ConvH = -ADVxy_diff['X']/vol #  x-partial derivative of zonal advection
adv_merid_ConvH= -ADVxy_diff['Y']/vol #  y-partial derivative of meridional advection
G_advection_conv_zon = adv_zonal_ConvH # Zonal advection convergence
G_advection_conv_mer = adv_merid_ConvH # Meridional advection convergence

In [ ]:
# Diffusion
dif_zonal_ConvH = -DFxyE_diff['X']/vol #  x-partial derivative of zonal diffusion
dif_merid_ConvH= -DFxyE_diff['Y']/vol #  y-partial derivative of zonal diffusion
G_diffusion_conv_zon = dif_zonal_ConvH # Zonal diffusion convergence
G_diffusion_conv_mer = dif_merid_ConvH # Meridional diffusion convergence

## Save to dataset
Now that we have all the terms evaluated, let's save them to a dataset. Here are two examples:
- Zarr is a new format that is used for cloud storage.
- Netcdf is the more traditional format that most people are familiar with.

When saving this heat budget dataset, the zarr file is ~15 GB, while the NetCDF file is ~53 GB. So zarr can be more efficient for storage.

### Add all variables to a new dataset

In [ ]:
def var_names_2ds(varnames):
    ds = xr.Dataset(data_vars={})
    #print(varnames)
    for varname in varnames:
        # Remove _box if there to focus on the variable name and the grid
        varname_base = varname.replace('_box', '')
        if varname_base in ['adv_vH', 'dif_vH']:
            ds[varname_base] = globals()[varname].chunk(chunks={'time':1,'tile':13,'k_l':50,'j':90,'i':90})
        elif varname_base in ['adv_zonalH', 'dif_zonalH']:
            ds[varname_base] = globals()[varname].chunk(chunks={'time':1,'tile':13,'k':50,'j':90,'i_g':90})
        elif varname_base in ['adv_meridH', 'dif_meridH']:
            ds[varname_base] = globals()[varname].chunk(chunks={'time':1,'tile':13,'k':50,'j_g':90,'i':90})
        elif varname_base in ['ETAN']:
            ds[varname_base] = globals()[varname].chunk(chunks={'time':1,'tile':13, 'j':90,'i':90})
        elif varname_base in ['vol']:
            ds[varname_base] = globals()[varname].chunk(chunks={'tile':13,'k':50,'j':90,'i':90})
        elif varname_base in ['XC_lon', 'YC_lat', 'area', 'geoflx_llc']:
            ds[varname_base] = globals()[varname].chunk(chunks={'tile':13,'j':90,'i':90})
        elif varname_base in ['XG_lon', 'YG_lat']:
            ds[varname_base] = globals()[varname].chunk(chunks={'tile':13,'j_g':90,'i_g':90})
        elif varname_base in ['Z_depth']:
            ds[varname_base] = globals()[varname].chunk(chunks={'k':50})
        else:
            ds[varname_base] = globals()[varname].chunk(chunks={'time':1,'tile':13,'k':50,'j':90,'i':90})
    return(ds)


In [ ]:
# Define which variables to save
varnames = ['G_total', 'G_advection', 'G_diffusion', 'G_forcing', \
            'adv_hConv', 'adv_vConv', 'dif_hConv', 'dif_vConv', \
           'G_advection_conv_zon', 'G_advection_conv_mer', \
           'G_diffusion_conv_zon', 'G_diffusion_conv_mer', \
           'DATA']
# varnames = ['DATA']

In [ ]:
# Create dataset with stored variables
ds = var_names_2ds(varnames)

In [ ]:
# varnames = ['G_total', 'G_advection', 'G_diffusion', 'G_forcing', \
#             'adv_hConv', 'adv_vConv', 'dif_hConv', 'dif_vConv']

# ds = xr.Dataset(data_vars={})
# for varname in varnames:
#     ds[varname] = globals()[varname].chunk(chunks={'time':1,'tile':13,'k':50,'j':90,'i':90})

In [ ]:
# Add surface forcing (degC/s)
ds['Qnet'] = ((forcH /(rhoconst*c_p))\
              /(ecco_grid.hFacC*ecco_grid.drF)).chunk(chunks={'time':1,'tile':13,'k':50,'j':90,'i':90})

In [ ]:
# Add shortwave penetrative flux (degC/s)
#Since we only are interested in the subsurface heat flux we need to zero out the top cell
SWpen = ((forcH_subsurf /(rhoconst*c_p))/(ecco_grid.hFacC*ecco_grid.drF)).where(forcH_subsurf.k>0).fillna(0.)
ds['SWpen'] = SWpen.where(ecco_grid.hFacC>0).chunk(chunks={'time':1,'tile':13,'k':50,'j':90,'i':90})

> **Note**: `Qnet` and `SWpen` are included in `G_forcing` and are not necessary to close the heat budget. 

In [ ]:
ds.time.encoding = {}
ds = ds.reset_coords(drop=True)

In [ ]:
# DEPTH VALUES
# array([-5.000000e+00, -1.500000e+01, -2.500000e+01, -3.500000e+01,
#        -4.500000e+01, -5.500000e+01, -6.500000e+01, -7.500500e+01,
#        -8.502500e+01, -9.509500e+01, -1.053100e+02, -1.158700e+02,
#        -1.271500e+02, -1.397400e+02, -1.544700e+02, -1.724000e+02,
#        -1.947350e+02, -2.227100e+02, -2.574700e+02, -2.999300e+02,
#        -3.506800e+02, -4.099300e+02, -4.774700e+02, -5.527100e+02,
#        -6.347350e+02, -7.224000e+02, -8.144700e+02, -9.097400e+02,
#        -1.007155e+03, -1.105905e+03, -1.205535e+03, -1.306205e+03,
#        -1.409150e+03, -1.517095e+03, -1.634175e+03, -1.765135e+03,
#        -1.914150e+03, -2.084035e+03, -2.276225e+03, -2.491250e+03,
#        -2.729250e+03, -2.990250e+03, -3.274250e+03, -3.581250e+03,
#        -3.911250e+03, -4.264250e+03, -4.640250e+03, -5.039250e+03,
#        -5.461250e+03, -5.906250e+03], dtype=float32)

### Save as netcdf for the first 10 depth levels

In [ ]:
# "cut" terms until the 10th pressure level
G_total_cut = ds.G_total.sel(k=slice(None, 9))
G_advection_conv_cut = ds.G_advection.sel(k=slice(None, 9))
G_diffusion_conv_cut = ds.G_diffusion.sel(k=slice(None, 9))
G_forcing_cut = ds.G_forcing.sel(k=slice(None, 9))
Qnet_cut = ds.Qnet.sel(k=slice(None, 9))
SWpen_cut = ds.SWpen.sel(k=slice(None, 9))
adv_hConv_cut = ds.adv_hConv.sel(k=slice(None, 9))
adv_vConv_cut = ds.adv_vConv.sel(k=slice(None, 9))
dif_hConv_cut = ds.dif_hConv.sel(k=slice(None, 9))
dif_vConv_cut = ds.dif_vConv.sel(k=slice(None, 9))
G_advection_conv_zon_cut = ds.G_advection_conv_zon.sel(k=slice(None, 9))
G_advection_conv_mer_cut = ds.G_advection_conv_mer.sel(k=slice(None, 9))
G_diffusion_conv_zon_cut = ds.G_diffusion_conv_zon.sel(k=slice(None, 9))
G_diffusion_conv_mer_cut = ds.G_diffusion_conv_mer.sel(k=slice(None, 9))
DATA_cut = ds.DATA.sel(k=slice(None, 9))

In [ ]:
# Define the variables to be saved
varnames = ['G_total_cut','G_advection_conv_cut','G_diffusion_conv_cut','G_forcing_cut',\
            'adv_hConv_cut','adv_vConv_cut','dif_hConv_cut','dif_vConv_cut', \
           'G_advection_conv_zon_cut', 'G_advection_conv_mer_cut', \
            'G_diffusion_conv_zon_cut', 'G_diffusion_conv_mer_cut', \
           'DATA_cut']

ds = xr.Dataset(data_vars={})
for varname in varnames:
    ds[varname] = globals()[varname].chunk(chunks={'time':1,'tile':13,'k':9,'j':90,'i':90})

In [ ]:
# Save the variables using netcdf
save_dir = join(ECCO_dir,'outputs')
for ivar_2save in varnames:
    ds_new = ds[ivar_2save]
    print(ivar_2save)
    ds_new.to_netcdf(save_dir + '/single_heat_terms/' + folder_year + '/ECCOv4r4_' + ivar_2save + '_' + folder_year + '.nc', format='NETCDF4')
    

In [ ]:
ciao

### Save to zarr

In [ ]:
# setting up a dask LocalCluster
from dask.distributed import Client
from dask.distributed import LocalCluster
cluster = LocalCluster()
client = Client(cluster)
client

In [ ]:
from dask.diagnostics import ProgressBar

In [ ]:
# save_dir is set to ~/Downloads below;
# change if you want to save somewhere else
save_dir = join(ECCO_dir,'outputs')
with ProgressBar():
    ds.to_zarr(join(save_dir,'eccov4r4_budg_heat_' + folder_year))

In [ ]:
### Save to netcdf

In [ ]:
# save_dir = join(ECCO_dir,'outputs')
# with ProgressBar():
#     ds.to_netcdf(join(save_dir,'eccov4r4_budg_heat.nc'), format='NETCDF4')

In [ ]:
# for ivar_2save in varnames:
#     ds_new = ds[ivar_2save]
#     ds_new.to_netcdf(save_dir + '/ECCOv4r4_' + ivar_2save + '_' + str(year_start) + '_' + str(year_end) + '.nc', format='NETCDF4')
    

### Re-load from zarr and save as netcdf for the first 10 depth levels

In [ ]:
# Load terms from zarr dataset
G_total = xr.open_zarr(join(save_dir,'eccov4r4_budg_heat_' + folder_year)).G_total
G_advection = xr.open_zarr(join(save_dir,'eccov4r4_budg_heat_' + folder_year)).G_advection
G_diffusion = xr.open_zarr(join(save_dir,'eccov4r4_budg_heat_' + folder_year)).G_diffusion
G_forcing = xr.open_zarr(join(save_dir,'eccov4r4_budg_heat_' + folder_year)).G_forcing
Qnet = xr.open_zarr(join(save_dir,'eccov4r4_budg_heat_' + folder_year)).Qnet
SWpen = xr.open_zarr(join(save_dir,'eccov4r4_budg_heat_' + folder_year)).SWpen

In [ ]:
# "cut" temrs until the 10th pressure level
G_total_cut = G_total.sel(k=slice(None, 9))
G_advection_cut = G_advection.sel(k=slice(None, 9))
G_diffusion_cut = G_diffusion.sel(k=slice(None, 9))
G_forcing_cut = G_forcing.sel(k=slice(None, 9))
Qnet_cut = Qnet.sel(k=slice(None, 9))
SWpen_cut = SWpen.sel(k=slice(None, 9))

In [ ]:
varnames = ['G_total_cut','G_advection_cut','G_diffusion_cut','G_forcing_cut']

ds = xr.Dataset(data_vars={})
for varname in varnames:
    ds[varname] = globals()[varname].chunk(chunks={'time':1,'tile':13,'k':9,'j':90,'i':90})

In [ ]:
for ivar_2save in varnames:
    ds_new = ds[ivar_2save]
    ds_new.to_netcdf(save_dir + '/single_heat_terms/' + folder_year + '/ECCOv4r4_' + ivar_2save + '_' + folder_year + '.nc', format='NETCDF4')
    

In [ ]:
ciao

In [ ]:
G_forcing_cut_test = G_forcing_cut*(G_total_cut/G_total_cut)

In [ ]:
G_forcing_cut_test[1,1,1,60,:].values

In [ ]:
G_forcing_cut[1,1,1,60,:].values

In [ ]:
G_total_cut[1,1,1,60,:].values

In [ ]:
G_forcing_cut[1,1,1,:,:].plot()

In [ ]:
ciao

## Load budget variables from file
After having saved the budget terms to file, we can load the dataset like this

In [ ]:
# Load terms from zarr dataset
G_total = xr.open_zarr(join(save_dir,'eccov4r4_budg_heat')).G_total
G_advection = xr.open_zarr(join(save_dir,'eccov4r4_budg_heat')).G_advection
G_diffusion = xr.open_zarr(join(save_dir,'eccov4r4_budg_heat')).G_diffusion
G_forcing = xr.open_zarr(join(save_dir,'eccov4r4_budg_heat')).G_forcing
Qnet = xr.open_zarr(join(save_dir,'eccov4r4_budg_heat')).Qnet
SWpen = xr.open_zarr(join(save_dir,'eccov4r4_budg_heat')).SWpen

In [ ]:
G_forcing

Or if you saved it as a netcdf file:
``` python
# Load terms from netcdf file
G_budget = xr.open_mfdataset(join(save_dir,'eccov4r4_budg_heat.nc'))
G_total_tendency = G_budget.G_total_tendency
G_advection = G_budget.G_advection
G_diffusion = G_budget.G_diffusion
G_forcing = G_budget.G_forcing
Qnet = G_budget.Qnet
```

## Comparison between LHS and RHS of the budget equation

In [ ]:
# Total convergence
ConvH = G_advection + G_diffusion

In [ ]:
# Sum of terms in RHS of equation
rhs = ConvH + G_forcing

### Map of residuals

In [ ]:
res = ((rhs-G_total).sum(dim='k')*month_length_weights).sum(dim='time').compute()

In [ ]:
if do_plots:
    plt.figure(figsize=(15,5))
    ecco.plot_proj_to_latlon_grid(ecco_grid.XC, ecco_grid.YC, res,
                                  cmin=-5e-12, cmax=5e-12, show_colorbar=True, cmap='RdBu_r',dx=0.2, dy=0.2)
    plt.title(r'Residual $\partial \theta / \partial t$ [$^\circ$C s$^{-1}$]: RHS - LHS', fontsize=16)
    plt.show()

The residual (summed over depth and time) is essentially zero everywhere. What if we omit the geothermal heat flux?

In [ ]:
# Residual when omitting geothermal heat flux
res_geo = ((ConvH + Qnet - G_total).sum(dim='k')*month_length_weights).sum(dim='time').compute()

In [ ]:
if do_plots:
    plt.figure(figsize=(15,5))
    ecco.plot_proj_to_latlon_grid(ecco_grid.XC, ecco_grid.YC, res_geo,
                                  cmin=-1e-9, cmax=1e-9, show_colorbar=True, cmap='RdBu_r', dx=0.2, dy=0.2)
    plt.title(r'Residual due to omitting geothermal heat [$^\circ$C s$^{-1}$] ', fontsize=16)
    plt.show()

We see that the contribution from geothermal flux in the heat budget is well above the residual (by *three orders of magnitude*).

In [ ]:
# Residual when omitting shortwave penetrative heat flux
res_sw = (rhs-SWpen-G_total).sum(dim='k').sum(dim='time').compute()

In [ ]:
if do_plots:
    plt.figure(figsize=(15,5))
    ecco.plot_proj_to_latlon_grid(ecco_grid.XC, ecco_grid.YC, res_sw,
                                  cmin=-5e-4, cmax=5e-4, show_colorbar=True, cmap='RdBu_r', dx=0.2, dy=0.2)
    plt.title(r'Residual due to omitting shortwave penetrative heat flux [$^\circ$C s$^{-1}$] ', fontsize=16)
    plt.show()

In terms of subsurface heat fluxes, shortwave penetration represents a much larger heat flux compared to geothermal heat flux (by around *three orders of magnitude*).

### Histogram of residuals
We can look at the distribution of residuals to get a little more confidence.

In [ ]:
tmp = np.abs(res).values.ravel()

In [ ]:
if do_plots:
    plt.figure(figsize=(10,3));

    plt.hist(tmp[tmp > 0],np.linspace(0, 2.e-12, 200));
    plt.grid()

Summing residuals vertically and temporally yields < $10^{-12}$ $^\circ$C s$^{-1}$ for most grid points.

## Heat budget closure through time

### Global average budget closure

Another way of demonstrating heat budget closure is to show the global spatially-averaged `THETA` tendency terms 

In [ ]:
# Volume (m^3)
vol = (ecco_grid.rA*ecco_grid.drF*ecco_grid.hFacC).transpose('tile','k','j','i')

# Take volume-weighted mean of these terms
tmp_a=(G_total*vol).sum(dim=('k','i','j','tile'))/vol.sum()
tmp_b=(G_advection*vol).sum(dim=('k','i','j','tile'))/vol.sum()
tmp_c=(G_diffusion*vol).sum(dim=('k','i','j','tile'))/vol.sum()
tmp_d=(G_forcing*vol).sum(dim=('k','i','j','tile'))/vol.sum()
tmp_e=(rhs*vol).sum(dim=('k','i','j','tile'))/vol.sum()

# Result is five time series
tmp_a.dims

In [ ]:
if do_plots:
    fig, axs = plt.subplots(2, 2, figsize=(14,8))

    plt.sca(axs[0,0])
    tmp_a.plot(color='k',lw=2)
    tmp_e.plot(color='grey')
    axs[0,0].set_title(r'a. $G^\theta_{total}$ (black) / RHS (grey) [$^\circ$C s$^{-1}$]', fontsize=12)
    plt.grid()

    plt.sca(axs[0,1])
    tmp_b.plot(color='r')
    axs[0,1].set_title(r'b. $G^\theta_{advection}$ [$^\circ$C s$^{-1}$]', fontsize=12)
    plt.grid()

    plt.sca(axs[1,0])
    tmp_c.plot(color='orange')
    axs[1,0].set_title(r'c. $G^\theta_{diffusion}$ [$^\circ$C s$^{-1}$]', fontsize=12)
    plt.grid()

    plt.sca(axs[1,1])
    tmp_d.plot(color='b')
    axs[1,1].set_title(r'd. $G^\theta_{forcing}$ [$^\circ$C s$^{-1}$]', fontsize=12)
    plt.grid()
    plt.subplots_adjust(hspace = .5, wspace=.2)
    plt.suptitle('Global Heat Budget', fontsize=16);

When averaged over the entire ocean the ocean heat transport terms ($G^\theta_\textrm{advection}$ and $G^\theta_\textrm{diffusion}$)  have no net impact on $G^\theta_\textrm{total}$ (i.e., $\partial \theta / \partial t$).  This makes sense because $G^\theta_\textrm{advection}$ and $G^\theta_\textrm{diffusion}$ can only redistributes heat. Globally, $\theta$ can only change via $G^\theta_\textrm{forcing}$.

### Local heat budget closure

Locally we expect that heat divergence can impact $\theta$. This is demonstrated for a single grid point.

In [ ]:
# Pick any set of indices (tile, k, j, i) corresponding to an ocean grid point
t,k,j,i = (6,10,40,29)
print(t,k,j,i)

In [ ]:
tmp_a = G_total.isel(tile=t,k=k,j=j,i=i)
tmp_b = G_advection.isel(tile=t,k=k,j=j,i=i)
tmp_c = G_diffusion.isel(tile=t,k=k,j=j,i=i)
tmp_d = G_forcing.isel(tile=t,k=k,j=j,i=i)
tmp_e = rhs.isel(tile=t,k=k,j=j,i=i)

if do_plots:
    fig, axs = plt.subplots(2, 2, figsize=(14,8))

    plt.sca(axs[0,0])
    tmp_a.plot(color='k',lw=2)
    tmp_e.plot(color='grey')
    axs[0,0].set_title(r'a. $G^\theta_{total}$ (black) / RHS (grey) [$^\circ$C s$^{-1}$]', fontsize=12)
    plt.grid()

    plt.sca(axs[0,1])
    tmp_b.plot(color='r')
    axs[0,1].set_title(r'b. $G^\theta_{advection}$ [$^\circ$C s$^{-1}$]', fontsize=12)
    plt.grid()

    plt.sca(axs[1,0])
    tmp_c.plot(color='orange')
    axs[1,0].set_title(r'c. $G^\theta_{diffusion}$ [$^\circ$C s$^{-1}$]', fontsize=12)
    plt.grid()

    plt.sca(axs[1,1])
    tmp_d.plot(color='b')
    axs[1,1].set_title(r'd. $G^\theta_{forcing}$ [$^\circ$C s$^{-1}$]', fontsize=12)
    plt.grid()
    plt.subplots_adjust(hspace = .5, wspace=.2)
    plt.suptitle('Heat Budget for one grid point (tile = %i, k = %i, j = %i, i = %i)'%(t,k,j,i), fontsize=16);

Indeed, the heat divergence terms do contribute to $\theta$ variations at a single point. Local heat budget closure is also confirmed at this grid point as we see that the sum of terms on the RHS (grey line) equals the LHS (black line).

For the Arctic grid point, there is a clear seasonal cycles in both $G^\theta_\textrm{total}$, $G^\theta_\textrm{diffusion}$ and $G^\theta_\textrm{forcing}$. The seasonal cycle in $G^\theta_\textrm{forcing}$ seems to be the reverse of $G^\theta_\textrm{total}$ and $G^\theta_\textrm{diffusion}$.

In [ ]:
if do_plots:
    plt.figure(figsize=(10,6));
    tmp_a.groupby('time.month').mean('time').plot(color='k',lw=3)
    tmp_b.groupby('time.month').mean('time').plot(color='r')
    tmp_c.groupby('time.month').mean('time').plot(color='orange')
    tmp_d.groupby('time.month').mean('time').plot(color='b')
    tmp_e.groupby('time.month').mean('time').plot(color='grey')
    plt.ylabel(r'$\partial\theta$/$\partial t$ [$^\circ$C s$^{-1}$]', fontsize=12)
    plt.grid()
    plt.title('Climatological seasonal cycles', fontsize=14)
    plt.show()

The mean seasonal cycle of the total is a balance between advection and diffusion. However, this is likely depth-dependent. How does the balance look across the upper 200 meter at that location?

## Time-mean vertical profiles

In [ ]:
if do_plots:
    fig = plt.subplots(1, 2, sharey=True, figsize=(12,7))

    plt.subplot(1, 2, 1)
    plt.plot(G_total.isel(tile=t,j=j,i=i).mean('time'), ecco_grid.Z,
             lw=4, color='black', marker='.', label=r'$G^\theta_{total}$ (LHS)')

    plt.plot(G_advection.isel(tile=t,j=j,i=i).mean('time'), ecco_grid.Z, 
             lw=2, color='red', marker='.', label=r'$G^\theta_{advection}$')

    plt.plot(G_diffusion.isel(tile=t,j=j,i=i).mean('time'), ecco_grid.Z, 
             lw=2, color='orange', marker='.', label=r'$G^\theta_{diffusion}$')

    plt.plot(G_forcing.isel(tile=t,j=j,i=i).mean('time'), ecco_grid.Z, 
             lw=2, color='blue', marker='.', label=r'$G^\theta_{forcing}$')
    plt.plot(rhs.isel(tile=t,j=j,i=i).mean('time'), ecco_grid.Z, lw=1, color='grey', marker='.', label='RHS')
    plt.xlabel(r'$\partial\theta$/$\partial t$ [$^\circ$C s$^{-1}$]', fontsize=14)
    plt.ylim([-200,0])
    plt.ylabel('Depth (m)', fontsize=14)
    plt.gca().tick_params(axis='both', which='major', labelsize=12)
    plt.legend(loc='lower left', frameon=False, fontsize=12)

    plt.subplot(1, 2, 2)
    plt.plot(G_total.isel(tile=t,j=j,i=i).mean('time'), ecco_grid.Z,
             lw=4, color='black', marker='.', label=r'$G^\theta_{total}$ (LHS)')
    plt.plot(rhs.isel(tile=t,j=j,i=i).mean('time'), ecco_grid.Z, lw=1, color='grey', marker='.', label='RHS')
    plt.setp(plt.gca(), 'yticklabels',[])
    plt.xlabel(r'$\partial\theta$/$\partial t$ [$^\circ$C s$^{-1}$]', fontsize=14)
    plt.ylim([-200,0])
    plt.show()

Balance between surface forcing and diffusion in the top layers. Balance between advection and diffusion at depth.